In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time
import os
import tkinter as tk
from tkinter import filedialog
from collections import deque

# === Constants and Settings ===
SCREEN_WIDTH, SCREEN_HEIGHT = 850, 480
HISTORY_LENGTH = 2
MIN_DRAWING_DISTANCE = 5
CLEAR_COOLDOWN = 1.0
MESSAGE_DURATION = 2.0
CLICK_DELAY = 0.1

# Gesture labels for help screen
GESTURE_LABELS = {
    "draw": "Only index finger open - Draws on canvas",
    "click": "Index, middle, and thumb open - Selects toolbar options",
    "eraser_mode": "Index and middle finger open - Activates eraser",
    "draw_mode": "Index, middle, ring fingers open - Activates draw mode",
    "change_theme": "All fingers open - Cycles canvas theme",
    "adjust_thickness": "Only thumb open - Adjusts brush/eraser size",
    "save": "Thumb and pinky open - Saves canvas as PNG",
    "load": "Middle, ring, pinky open - Loads an image",
    "clear": "All fingers closed - Clears the canvas"
}

# === Canvas Class ===
class DrawingCanvas:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.background = np.ones((height, width, 3), dtype=np.uint8) * 255
        self.drawing_layer = np.zeros((height, width, 4), dtype=np.uint8)
        self.image_layer = np.zeros((height, width, 4), dtype=np.uint8)
        self.current_theme = 'white'
        self.last_message = ""
        self.message_time = 0
        self.themes = ['white', 'gray', 'black']
        self.theme_index = 0
        self.show_help = False
        self.current_gesture = None
        self.gesture_time = 0
        
    def clear(self):
        self.drawing_layer.fill(0)
        
    def set_theme(self, theme):
        if theme == 'white':
            self.background.fill(255)
        elif theme == 'gray':
            self.background.fill(128)
        elif theme == 'black':
            self.background.fill(0)
        self.current_theme = theme
        self.show_message(f"Theme: {theme}")
        
    def cycle_theme(self):
        self.theme_index = (self.theme_index + 1) % len(self.themes)
        self.set_theme(self.themes[self.theme_index])
        
    def show_message(self, message):
        self.last_message = message
        self.message_time = time.time()
        
    def set_gesture(self, gesture):
        if gesture != self.current_gesture:
            self.current_gesture = gesture
            self.gesture_time = time.time()
        
    def get_display(self):
        if self.show_help:
            help_display = np.ones((self.height, self.width, 3), dtype=np.uint8) * 200
            cv2.putText(help_display, "Hand Drawing App - Help", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
            cv2.putText(help_display, "App Info:", (10, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1)
            cv2.putText(help_display, "Draw on a canvas using hand gestures or mouse.", (10, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
            cv2.putText(help_display, "Use toolbar or gestures to change colors, themes, or tools.", (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
            cv2.putText(help_display, "Gestures:", (10, 160),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1)
            for i, (gesture, desc) in enumerate(GESTURE_LABELS.items()):
                cv2.putText(help_display, f"{gesture}: {desc}", (10, 180 + i * 25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
            cv2.putText(help_display, "Click '?' again to return to canvas", (10, self.height - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
            return help_display
        
        display = self.background.copy()
        alpha = self.image_layer[:, :, 3] / 255.0
        for c in range(3):
            display[:, :, c] = (1 - alpha) * display[:, :, c] + alpha * self.image_layer[:, :, c]
        alpha = self.drawing_layer[:, :, 3] / 255.0
        for c in range(3):
            display[:, :, c] = (1 - alpha) * display[:, :, c] + alpha * self.drawing_layer[:, :, c]
        
        if time.time() - self.message_time < MESSAGE_DURATION:
            cv2.putText(display, self.last_message, (10, SCREEN_HEIGHT - 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
        if self.current_gesture and time.time() - self.gesture_time < MESSAGE_DURATION:
            cv2.putText(display, f"Gesture: {self.current_gesture}", (10, SCREEN_HEIGHT - 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
        return display

# === Gesture Recognizer ===
class GestureRecognizer:
    def __init__(self):
        self.hand_history = deque(maxlen=HISTORY_LENGTH)
        self.last_clear_time = 0
        self.last_action_time = 0
        self.clear_triggered = False
        self.save_triggered = False
        self.load_triggered = False
        self.was_palm_facing = None
        
    def update(self, hand_landmarks, hand_label):
        if hand_landmarks is None:
            self.hand_history.clear()
            self.clear_triggered = False
            self.save_triggered = False
            self.load_triggered = False
            self.was_palm_facing = None
            return None
            
        landmarks = []
        for lm in hand_landmarks.landmark:
            landmarks.append((lm.x * SCREEN_WIDTH, lm.y * SCREEN_HEIGHT))
        
        self.hand_history.append((landmarks, hand_label))
        
        if len(self.hand_history) < HISTORY_LENGTH:
            return None
            
        return self._analyze_gesture()
        
    def _analyze_gesture(self):
        current = self.hand_history[-1]
        landmarks, hand_label = current
        
        def is_finger_open(finger_tip, finger_pip):
            return finger_tip[1] < finger_pip[1]
            
        index_open = is_finger_open(landmarks[8], landmarks[6])
        middle_open = is_finger_open(landmarks[12], landmarks[10])
        ring_open = is_finger_open(landmarks[16], landmarks[14])
        pinky_open = is_finger_open(landmarks[20], landmarks[18])
        thumb_open = landmarks[4][1] < landmarks[2][1] - 0.03 * SCREEN_HEIGHT
        
        wrist = landmarks[0]
        middle_mcp = landmarks[9]
        is_palm_facing = None
        if hand_label == "Right":
            is_palm_facing = wrist[0] < middle_mcp[0]
        elif hand_label == "Left":
            is_palm_facing = wrist[0] > middle_mcp[0]
            
        current_time = time.time()
        
        if index_open and middle_open and not ring_open and not pinky_open and not thumb_open:
            return 'eraser_mode'
        
        if index_open and middle_open and ring_open and not pinky_open and not thumb_open:
            return 'draw_mode'
            
        all_fingers_open = all([thumb_open, index_open, middle_open, ring_open, pinky_open])
        if (all_fingers_open and self.was_palm_facing is not None and 
            self.was_palm_facing and not is_palm_facing and 
            current_time - self.last_action_time > CLEAR_COOLDOWN):
            self.was_palm_facing = is_palm_facing
            self.last_action_time = current_time
            return 'change_theme'
            
        self.was_palm_facing = is_palm_facing
        
        if thumb_open and not index_open and not middle_open and not ring_open and not pinky_open:
            self.clear_triggered = False
            return 'adjust_thickness'
            
        if thumb_open and pinky_open and not index_open and not middle_open and not ring_open:
            if current_time - self.last_action_time > CLEAR_COOLDOWN and not self.save_triggered:
                self.last_action_time = current_time
                self.save_triggered = True
                return 'save'
            return None
            
        self.save_triggered = False
        
        if not thumb_open and not index_open and middle_open and ring_open and pinky_open:
            if current_time - self.last_action_time > CLEAR_COOLDOWN and not self.load_triggered:
                self.last_action_time = time.time()
                self.load_triggered = True
                return 'load'
            return None
            
        self.load_triggered = False
        
        if not any([index_open, middle_open, ring_open, pinky_open, thumb_open]):
            if current_time - self.last_clear_time > CLEAR_COOLDOWN and not self.clear_triggered:
                self.last_clear_time = current_time
                self.clear_triggered = True
                self.last_action_time = current_time
                return 'clear'
                
        elif index_open and not middle_open and not ring_open and not pinky_open:
            self.clear_triggered = False
            return 'draw'
            
        elif index_open and middle_open and not ring_open and not pinky_open and thumb_open:
            self.clear_triggered = False
            return 'click'
            
        return None

# === Drawing Tools ===
class DrawingTool:
    def __init__(self):
        self.color = (0, 0, 0)
        self.thickness = 5
        self.alpha = 1.0
        self.is_eraser = False
        
    def set_color(self, color):
        self.color = color
        self.is_eraser = False
        
    def set_eraser(self, thickness=10):
        self.is_eraser = True
        self.thickness = thickness
        
    def draw(self, canvas, start_point, end_point):
        if self.is_eraser:
            mask = np.zeros((canvas.height, canvas.width), dtype=np.uint8)
            cv2.line(mask, start_point, end_point, 255, self.thickness)
            canvas.drawing_layer[mask > 0, 3] = 0
        else:
            temp_layer = np.zeros((canvas.height, canvas.width, 4), dtype=np.uint8)
            cv2.line(temp_layer, start_point, end_point, (*self.color, 255), self.thickness)
            alpha = temp_layer[:, :, 3] / 255.0 * self.alpha
            for c in range(3):
                canvas.drawing_layer[:, :, c] = (1 - alpha) * canvas.drawing_layer[:, :, c] + alpha * temp_layer[:, :, c]
            canvas.drawing_layer[:, :, 3] = np.maximum(canvas.drawing_layer[:, :, 3], temp_layer[:, :, 3] * self.alpha)

# === Main Application ===
class HandDrawingApp:
    def __init__(self):
        self.canvas = DrawingCanvas(SCREEN_WIDTH, SCREEN_HEIGHT)
        self.tool = DrawingTool()
        self.gesture_recognizer = GestureRecognizer()
        
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            max_num_hands=1,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.show_hand_status = True
        self.last_click_time = 0
        self.click_delay = CLICK_DELAY
        self.last_thickness_change = 0
        self.thickness_change_delay = 0.1
        
        self.last_point = None
        self.drawing = False
        self.mouse_drawing = False
        self.temp_layer = np.zeros((SCREEN_HEIGHT, SCREEN_WIDTH, 4), dtype=np.uint8)
        
        self.COLORS = {
            'black': (0, 0, 0), 'red': (0, 0, 255), 'green': (0, 255, 0),
            'blue': (255, 0, 0), 'yellow': (0, 255, 255), 'cyan': (255, 255, 0)
        }
        
        cv2.namedWindow('Canvas')
        cv2.namedWindow('Camera')
        cv2.setMouseCallback('Canvas', self.mouse_callback)
        cv2.setMouseCallback('Camera', self.camera_mouse_callback)
        
        self.show_camera = True
        self.show_canvas = True
    
    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.mouse_drawing = True
            self.last_point = (x, y)
            self._check_ui_click(x, y)
        elif event == cv2.EVENT_LBUTTONUP:
            self.mouse_drawing = False
            self.last_point = None
        elif event == cv2.EVENT_MOUSEMOVE and self.mouse_drawing and not self.canvas.show_help:
            if self.last_point:
                current_point = (x, y)
                distance = np.sqrt((current_point[0] - self.last_point[0])**2 + 
                                 (current_point[1] - self.last_point[1])**2)
                if distance > MIN_DRAWING_DISTANCE:
                    self.tool.draw(self.canvas, self.last_point, current_point)
                    self.last_point = current_point
    
    def camera_mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self._check_ui_click(x, y, is_camera=True)
    
    def _check_ui_click(self, x, y, is_camera=False):
        if y > 80:
            return
            
        current_time = time.time()
        if current_time - self.last_click_time < self.click_delay:
            return
            
        num_colors = len(self.COLORS)
        eraser_pos = 10 + num_colors * 60
        theme1_pos = eraser_pos + 60
        theme2_pos = theme1_pos + 60
        theme3_pos = theme2_pos + 60
        save_pos = theme3_pos + 60
        load_pos = save_pos + 60
        clear_pos = load_pos + 60
        help_pos = clear_pos + 60
        
        if help_pos <= x <= help_pos + 60:
            print("Help icon clicked at:", (x, y))
            self.canvas.show_help = not self.canvas.show_help
            self.last_click_time = current_time
            self.canvas.show_message("Help toggled" if self.canvas.show_help else "Returned to canvas")
            return
        
        if self.canvas.show_help or is_camera:
            return
            
        for i, color in enumerate(self.COLORS):
            x_start = 10 + i * 60
            x_end = x_start + 50
            if x_start <= x <= x_end:
                self.tool.set_color(self.COLORS[color])
                self.last_click_time = current_time
                self.canvas.show_message(f"Color: {color}")
                return
                
        if eraser_pos <= x <= eraser_pos + 50:
            self.tool.set_eraser(10)
            self.last_click_time = current_time
            self.canvas.show_message("Eraser active")
            return
            
        if theme1_pos <= x <= theme1_pos + 50:
            self.canvas.set_theme('white')
            self.last_click_time = current_time
            return
            
        if theme2_pos <= x <= theme2_pos + 50:
            self.canvas.set_theme('gray')
            self.last_click_time = current_time
            return
            
        if theme3_pos <= x <= theme3_pos + 50:
            self.canvas.set_theme('black')
            self.last_click_time = current_time
            return
            
        if save_pos <= x <= save_pos + 50:
            self.save_canvas()
            self.last_click_time = current_time
            return
            
        if load_pos <= x <= load_pos + 50:
            self.load_image()
            self.last_click_time = current_time
            return
            
        if clear_pos <= x <= clear_pos + 50:
            self.canvas.clear()
            self.canvas.show_message("Canvas cleared")
            self.last_click_time = current_time
            return
    
    def save_canvas(self):
        filename = f'drawing_{int(time.time())}.png'
        display = self.canvas.get_display()
        cv2.imwrite(filename, display)
        self.canvas.show_message(f"Saved as {filename}")
    
    def load_image(self):
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.gif *.tiff")]
        )
        if file_path:
            img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
            if img is not None:
                img = cv2.resize(img, (SCREEN_WIDTH, SCREEN_HEIGHT))
                if img.shape[2] == 3:
                    alpha_channel = np.ones((img.shape[0], img.shape[1]), dtype=np.uint8) * 255
                    img = np.dstack((img, alpha_channel))
                self.canvas.image_layer = img
                self.canvas.show_message(f"Loaded: {os.path.basename(file_path)}")
    
    def draw_toolbar(self, frame, index_tip_pos=None):
        num_colors = len(self.COLORS)
        eraser_pos = 10 + num_colors * 60
        theme1_pos = eraser_pos + 60
        theme2_pos = theme1_pos + 60
        theme3_pos = theme2_pos + 60
        save_pos = theme3_pos + 60
        load_pos = save_pos + 60
        clear_pos = load_pos + 60
        help_pos = clear_pos + 60
        
        is_help_hovered = False
        if index_tip_pos and help_pos <= index_tip_pos[0] <= help_pos + 60 and index_tip_pos[1] <= 80:
            is_help_hovered = True
            
        for i, color in enumerate(self.COLORS):
            x = 10 + i * 60
            cv2.rectangle(frame, (x, 10), (x + 50, 60), self.COLORS[color], -1)
            if self.tool.color == self.COLORS[color] and not self.tool.is_eraser:
                cv2.rectangle(frame, (x, 10), (x + 50, 60), (0, 255, 0), 3)
        
        cv2.rectangle(frame, (eraser_pos, 10), (eraser_pos + 50, 60), (200, 200, 200), -1)
        cv2.putText(frame, "Erase", (eraser_pos + 5, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        if self.tool.is_eraser:
            cv2.rectangle(frame, (eraser_pos, 10), (eraser_pos + 50, 60), (0, 255, 0), 3)
        
        cv2.rectangle(frame, (theme1_pos, 10), (theme1_pos + 50, 60), (255, 255, 255), -1)
        cv2.putText(frame, "White", (theme1_pos + 5, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        cv2.rectangle(frame, (theme1_pos, 10), (theme1_pos + 50, 60), (0, 0, 0), 3)
        
        cv2.rectangle(frame, (theme2_pos, 10), (theme2_pos + 50, 60), (128, 128, 128), -1)
        cv2.putText(frame, "Gray", (theme2_pos + 5, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
        cv2.rectangle(frame, (theme3_pos, 10), (theme3_pos + 50, 60), (0, 0, 0), -1)
        cv2.putText(frame, "Black", (theme3_pos + 5, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        cv2.rectangle(frame, (save_pos, 10), (save_pos + 50, 60), (150, 150, 255), -1)
        cv2.putText(frame, "Save", (save_pos + 10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
        cv2.rectangle(frame, (load_pos, 10), (load_pos + 50, 60), (150, 255, 150), -1)
        cv2.putText(frame, "Load", (load_pos + 10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
        cv2.rectangle(frame, (clear_pos, 10), (clear_pos + 50, 60), (200, 200, 100), -1)
        cv2.putText(frame, "Clear", (clear_pos + 10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
        cv2.rectangle(frame, (help_pos, 10), (help_pos + 60, 60), (100, 100, 255), -1)
        cv2.putText(frame, "?", (help_pos + 20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        if self.canvas.show_help or is_help_hovered:
            cv2.rectangle(frame, (help_pos, 10), (help_pos + 60, 60), (0, 255, 0), 3)
        
        indicator_pos = help_pos + 80
        if self.tool.is_eraser:
            cv2.circle(frame, (indicator_pos, 35), self.tool.thickness // 2, (0, 0, 0), -1)
            cv2.circle(frame, (indicator_pos, 35), self.tool.thickness // 2, (0, 0, 255), 1)
        else:
            cv2.circle(frame, (indicator_pos, 35), self.tool.thickness // 2, self.tool.color, -1)
            cv2.circle(frame, (indicator_pos, 35), self.tool.thickness // 2, (255, 255, 255), 1)
    
    def run(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            self.canvas.show_message("Error: Could not open camera")
            return
            
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, SCREEN_WIDTH)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, SCREEN_HEIGHT)
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    self.canvas.show_message("Error: Could not read frame")
                    break
                    
                frame = cv2.flip(frame, 1)
                display_frame = frame.copy()
                
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = self.hands.process(rgb)
                
                hand_label = None
                index_tip_pos = None
                current_gesture = None
                
                if results.multi_hand_landmarks:
                    hand_landmarks = results.multi_hand_landmarks[0]
                    self.mp_drawing.draw_landmarks(
                        display_frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
                    
                    index_tip = hand_landmarks.landmark[8]
                    index_tip_pos = (int(index_tip.x * SCREEN_WIDTH), int(index_tip.y * SCREEN_HEIGHT))
                    
                    if results.multi_handedness:
                        hand_label = results.multi_handedness[0].classification[0].label
                
                gesture = self.gesture_recognizer.update(
                    results.multi_hand_landmarks[0] if results.multi_hand_landmarks else None,
                    hand_label
                )
                
                if gesture:
                    self.canvas.set_gesture(gesture)
                    current_gesture = gesture
                
                if index_tip_pos:
                    cv2.circle(display_frame, index_tip_pos, 8, (0, 0, 255), -1)
                    cv2.circle(display_frame, index_tip_pos, 6, (255, 255, 255), 2)
                
                if gesture == 'eraser_mode' and not self.canvas.show_help:
                    self.tool.set_eraser(10)
                    self.canvas.show_message("Eraser mode activated")
                elif gesture == 'draw_mode' and not self.canvas.show_help:
                    self.tool.set_color(self.tool.color)
                    self.canvas.show_message("Draw mode activated")
                
                if gesture == 'click' and not self.mouse_drawing and index_tip_pos:
                    self.canvas.show_message("Click gesture detected")
                    if index_tip_pos[1] < 80:
                        self._check_ui_click(index_tip_pos[0], index_tip_pos[1])
                    self.last_point = None
                    self.drawing = False
                
                if not self.canvas.show_help:
                    if gesture == 'draw' and not self.mouse_drawing and index_tip_pos:
                        current_point = index_tip_pos
                        
                        if self.last_point:
                            distance = np.sqrt(
                                (current_point[0] - self.last_point[0])**2 + 
                                (current_point[1] - self.last_point[1])**2)
                            if distance > MIN_DRAWING_DISTANCE:
                                self.tool.draw(self.canvas, self.last_point, current_point)
                                self.last_point = current_point
                                self.drawing = True
                        else:
                            self.last_point = current_point
                    elif gesture != 'draw' and self.drawing:
                        self.last_point = None
                        self.drawing = False
                    
                    elif gesture == 'clear':
                        self.canvas.clear()
                        self.canvas.show_message("Canvas cleared")
                    elif gesture == 'adjust_thickness' and results.multi_hand_landmarks:
                        current_time = time.time()
                        if current_time - self.last_thickness_change > self.thickness_change_delay:
                            thumb_tip = results.multi_hand_landmarks[0].landmark[4]
                            thumb_direction = thumb_tip.y - results.multi_hand_landmarks[0].landmark[3].y
                            if thumb_direction < -0.02:
                                self.tool.thickness = min(20, self.tool.thickness + 1)
                                self.canvas.show_message(f"Size: {self.tool.thickness}")
                            elif thumb_direction > 0.01:
                                self.tool.thickness = max(1, self.tool.thickness - 1)
                                self.canvas.show_message(f"Size: {self.tool.thickness}")
                            self.last_thickness_change = current_time
                    elif gesture == 'change_theme':
                        self.canvas.cycle_theme()
                    elif gesture == 'save':
                        self.save_canvas()
                    elif gesture == 'load':
                        self.load_image()
                
                self.draw_toolbar(display_frame, index_tip_pos)
                
                canvas_display = self.canvas.get_display()
                
                if index_tip_pos and not self.canvas.show_help:
                    cv2.circle(canvas_display, index_tip_pos, 8, (0, 0, 255), -1)
                    cv2.circle(canvas_display, index_tip_pos, 6, (255, 255, 255), 2)
                
                if not self.canvas.show_help:
                    self.draw_toolbar(canvas_display, index_tip_pos)
                
                # Kamera penceresindeki yazıları sol alt köşeye taşıyoruz
                if self.show_hand_status:
                    status = "Hand detected" if results.multi_hand_landmarks else "No hand detected"
                    color = (0, 255, 0) if results.multi_hand_landmarks else (0, 0, 255)
                    cv2.putText(display_frame, status, (10, SCREEN_HEIGHT - 90), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                if current_gesture and time.time() - self.canvas.gesture_time < MESSAGE_DURATION:
                    cv2.putText(display_frame, f"Gesture: {current_gesture}", (10, SCREEN_HEIGHT - 60), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                if time.time() - self.canvas.message_time < MESSAGE_DURATION:
                    cv2.putText(display_frame, self.canvas.last_message, (10, SCREEN_HEIGHT - 30), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                if self.canvas.show_help:
                    cv2.imshow('Camera', display_frame)
                    cv2.imshow('Canvas', canvas_display)
                else:
                    if self.show_camera:
                        cv2.imshow('Camera', display_frame)
                    else:
                        cv2.destroyWindow('Camera')
                    
                    if self.show_canvas:
                        cv2.imshow('Canvas', canvas_display)
                    else:
                        cv2.destroyWindow('Canvas')
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('h'):
                    self.show_hand_status = not self.show_hand_status
                elif key == ord('s') and not self.canvas.show_help:
                    self.save_canvas()
                elif key == ord('l') and not self.canvas.show_help:
                    self.load_image()
                elif key == ord('r') and not self.canvas.show_help:
                    self.canvas.clear()
                    self.canvas.show_message("Canvas cleared")
                elif key == ord('+') and not self.canvas.show_help:
                    self.tool.thickness = min(20, self.tool.thickness + 1)
                    self.canvas.show_message(f"Size: {self.tool.thickness}")
                elif key == ord('-') and not self.canvas.show_help:
                    self.tool.thickness = max(1, self.tool.thickness - 1)
                    self.canvas.show_message(f"Size: {self.tool.thickness}")
                elif key == ord('?'):
                    self.canvas.show_help = not self.canvas.show_help
                    self.canvas.show_message("Help toggled" if self.canvas.show_help else "Returned to canvas")
                elif key == ord('c'):
                    self.show_camera = not self.show_camera
                    if not self.show_camera:
                        cv2.destroyWindow('Camera')
                
        finally:
            cap.release()
            cv2.destroyAllWindows()
            self.hands.close()

if __name__ == "__main__":
    app = HandDrawingApp()
    app.run()

Help icon clicked at: (799, 48)
Help icon clicked at: (792, 16)
Help icon clicked at: (801, 26)
Help icon clicked at: (801, 26)
